## Data Preparation
First we import our necessary libraries

In [ ]:
%%script echo skipping # Comment this line out if you want to install these packages
!pip install pygris
!pip install folium

In [11]:
# Load packages
%load_ext autoreload
%autoreload 2
import pygris
from pygris import tracts
from matplotlib import pyplot as plt
import pandas as pd
import folium
import numpy as np
from linear_regression import RidgeRegression
import geopandas as gpd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Starting up 
As a test of concept, lets utilize the pygris library to access the CT tracts information and then let's do a simple plot to ensure it's correct.

In [ ]:
# Download geometry
ct_tracts = tracts(state = "CT", cb = True, cache = True, year = 2016)

# Display geometry
fig, ax = plt.subplots()
ct_tracts.plot(ax = ax)
plt.title("Tracts Cartographic Boundaries");

## Graphing Pop-Density
Before we begin our journey into zonal statistics and eventually creating a predictive model, we first want to understand what the population density looks like in Connecticut. We have some general hypotheses that the areas around New Haven and Hartford are going to have higher amounts of population, and we also expect to see some small pockets of communities around Connecticut.

In [ ]:
# Import tracts population data
pop = pd.read_csv("../data/population.csv")

# Convert data type so join key matches
ct_tracts["Geo_TRACT"] = ct_tracts["TRACTCE"].astype(int)

# Join attributes to geometry
tracts = ct_tracts.merge(pop, how = "inner", on='Geo_TRACT')

### Once we have our tracts, we then calculate pop density and graph 

In [ ]:
# Project tracts
tracts = tracts.to_crs("EPSG:3857")

# Calculate area in KM
tracts["Area"] = tracts.area/1000**2

# Calculate population density
tracts["PopDensity"] = tracts["SE_A00001_001"]/tracts["Area"]

In [ ]:
tracts.plot("PopDensity", legend = True);

## **maybe here we add some enlightening information about our tracts dataframe, otherwise we can remove the code cell above**

## Intro to Zonal Statistics
We are going to do quite a bit here which is being abstracted away by the [`rasterio`](https://pypi.org/project/rasterio/) package. First and foremost, we are going to load in our landcover data from [CONUS](https://www.ncei.noaa.gov/products/satellite/gridded-goes-conus) which is created from satellite date of the Continental United States.

This data comes in the form of a `.tif` file which is a filetype used for storing geographic satellite data.

The goal of zonal statistics here is relatively straightforward, we are going to do some math that involves the pixels in a given geographic satellite image. Each pixel has an associated number which itself is associated with a key. Each pixel is contained in a "tract" which is a measurement of land by the US Census. We perform mathematics like finding the mean type of pixel in a given area, the max, the minimum, etc. This arithmetic is handled by the [`rasterstats`](https://pythonhosted.org/rasterstats/) package.

In [ ]:
!pip install rasterio
import rasterio

### First steps 
Here we open our path to our file, and more importantly, we set up our data to be used in zonal statistics. `.read` turns our data into a Numpy Array. Following this we are going to `.transform` our data, which means we are going to take the pixel locations of our coordinates (row col) and map them to our spatial coordinates (x, y). These coordinate values are relative to the [CRS](https://www.ncei.noaa.gov/products/satellite/gridded-goes-conus) (Coordinate Reference System) which we defined earlier as **"EPSG:2234"**

In [ ]:
#the data can be accessed from https://coastalimagery.blob.core.windows.net/ccap-landcover/CCAP_bulk_download/High_Resolution_Land_Cover/Phase_2_Expanded_Categories/Legacy_Land_Cover_pre_2024/CONUS/ct_2016_ccap_hires_landcover_20200915.zip
raster_path = '../data/ct_2016_ccap_hires_landcover_20200915.tif'
landcover = rasterio.open(raster_path)
arr = landcover.read(1)
affine = landcover.transform

### Performing Zonal statistics
It's as simple as importing rasterstats. We have handled the important data manipulation, and now it's basically plug and play! One function to note is `.to_crs` which takes in given coordinate reference system and transforms all the points in our dataframe to match that system.

In [ ]:
#!pip install rasterstats

from rasterstats import zonal_stats

# Here is where we ensure our tracts CRS is the same as our landcover CRS
zone = tracts.to_crs(landcover.crs)
stats = zonal_stats(zone, arr, affine=affine)

## BOOM 
We did it! Taking a look at our dataframe we can see what this basic zonal statistics calculated the min, max, mean, and amount of pixels in the tract

In [ ]:
df = pd.DataFrame(stats)
df

## Advanced Zonal Statistics
The `rasterstats` library is very good at getting information from rasters, and we can in fact gain more information by using `categorical = True`. This allows to see the amount of each type of pixel at a given tract.

In [ ]:
from rasterstats import zonal_stats
df_new = zonal_stats(zone, arr, affine=affine, categorical = True)

Taking a look at our dataframe, we can confirm that each column is a type of pixel and each row is a tract

In [ ]:
df_categorical = pd.DataFrame(df_new)
df_categorical

## Visualizing Zonal Stats
Now that we have information on the amount of each pixel at a given tract, we can find the most common pixel per tract by using the function `.idxmax()` which will through each row and find the column with the largest value.

In [ ]:
df_categorical['max_type'] = df_categorical.idxmax(axis=1)
combined_df = pd.concat([tracts, df_categorical], axis=1)
combined_df['max_type'] = combined_df['max_type'].astype(str)

In [ ]:
combined_df.plot("max_type", legend = True);

In [ ]:
combined_df["geometry"]

### Saving this data
These statistics took quite a while to run, and it may be beneficial to save this data as a csv to continue running statistics in the future

In [ ]:
%%%script echo skipping

combined_df.to_csv('../data/combined_data.csv', index=False)

## Fitting a Regression Model

First, we import our data.

In [8]:
# Import and display data
data = pd.read_csv("combined_data.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'combined_data.csv'

Looks like there is some missing data in tracts that contain no pixels of a certain class.
Let's impute 0 for all `NaN` values.

In [9]:
# Impute 0 for missing data
print("Before imputation, there were", pd.isnull(data.iloc[:,68:-1]).sum().sum(), "NaN values.")
data[pd.isnull(data.iloc[:,68:-1])] = 0
print("After imputation, there are", pd.isnull(data.iloc[:,68:-1]).sum().sum(), "NaN values.")
data.head()

NameError: name 'data' is not defined

Now that we have complete data, we can calculate the proportion of pixels belonging to each class.

In [ ]:
# Calculate total number of pixels in each tract
data["sum"] = data.iloc[:,68:-1].sum(axis = 1)

# Calculate proportion of pixels belonging to each class
data.iloc[:,68:-2] = data.iloc[:,68:-2].div(data['sum'], axis=0)

# View data
data.head()

In [ ]:
# Separate predictors and outcome
X = data.iloc[:,68:-2]
y = data["PopDensity"]

In [ ]:
# Fit model
# Doing this just for the purpose of seeing what it looks like
# We can use the results from this package to verify that our implementation is working properly
from sklearn.linear_model import LinearRegression
LR = LinearRegression()

m = LR.fit(X, y)

In [ ]:
# R^2 value
m.score(X, y)

In [ ]:
# Y-intercept
m.intercept_

In [ ]:
# Variable coefficients
m.coef_

In [ ]:
# Create predictions (probably should do this on a test set)
preds = LR.predict(X)

In [ ]:
data["PopDensity"]

In [ ]:
data

It would be cool to make a map of the predictions and have it next to a map of the actual values.

In [6]:
RR = RidgeRegression()
RR.fit(X, y)

# To do
- move `combined_data.csv` to data directory
- set raster processing cells not to run by default
- perform cross-validation or create training and test data sets
- maybe save data as a shp rather than a csv?